# 当前问题
1. 载重量中的炉水量由于没有锅炉额定蒸发量的信息，目前取0
2. 固定压载和排水量裕度问题暂时没有考虑，参考课本p66
3. 重量校核结果显示重量大于排水量，我们现在是从增加吃水的角度来解决的，因此主尺度是不变的，无需再次校核，仅仅是载重量多了船多了些吃水

# 排水量估计

对于载重型船舶初步的排水量可用型船的载重量系数来估算：  
$$\triangle = \frac{DW}{\eta_{DW}}$$

## 母船资料

In [1]:
DW_parent = 75088 # 母船载重量，t
nabla_parent = 85488.56 # 母船结构吃水排水体积
delta_parent = 87809.16 # 母船结构吃水排水量
rho = 1.025
eta_DW_parent = round(DW_parent/delta_parent,3)
print(f'''
母船载重量        {DW_parent} t
母船排水量        {delta_parent} t
母船载重量系数    {eta_DW_parent}
''')


母船载重量        75088 t
母船排水量        87809.16 t
母船载重量系数    0.855



In [2]:
# 母船主尺度
Lpp_parent = 218
B_parent = 32.26
d_structure_parent = 14.3
D_parent = 19.6
names = ["长","型宽","结构吃水","型深"]
values = [Lpp_parent,B_parent,d_structure_parent,D_parent]
print("母船主尺度数据:")
for n,v in zip(names,values):
    print(n.ljust(20)+f'{v}m')

母船主尺度数据:
长                   218m
型宽                  32.26m
结构吃水                14.3m
型深                  19.6m


## 设计船排水量

In [3]:
DW = 72100
delta = round(DW/eta_DW_parent,2)
Lpp = 215.17
B = 31.84
d = 14.11
D = 19.34
Vs = 14.5 # 根据设计任务书p3，服务航速为14.5kn
R = 24000 # 根据设计任务书p3，续航力取24000 n miles
print(f'''
设计船主尺度数据：
设计船载重量为{DW}t
设计船排水量为{delta}t
长{Lpp}m
型宽{B}m
吃水{d}m
型深{D}m
''')


设计船主尺度数据：
设计船载重量为72100t
设计船排水量为84327.49t
长215.17m
型宽31.84m
吃水14.11m
型深19.34m



# 重量估算

In [4]:
import numpy as np
import pandas as pd

**母船重量资料**

In [5]:
# 钢料重量
data_steel_parent = pd.read_excel('./data/母船钢料.xlsx')
data_steel_parent.head()

,ITEM,W,LCG,AFTEXT,FWD EXT
0,STF-6~F11+.3,260.21,3.59,-4,8.55
1,STF11+.3~24+.35,323.26,14.28,8.55,19.00
2,ST.F24+.35~F36+.3,507.47,24. 60,1900,28.55
3,STF36+.3~F64,820.69,40.78,28.55,52.61
4,ST. F64~F214,4889.31,119.23,52.61,183.11


In [6]:
data_outfitting_parent = pd.read_excel('./data/母船舾装与机电设备.xlsx')
data_outfitting_parent.head()

,ITEM,W,LCG,AFT EXT,FWD EXT,类别,系数
0,F'CLE,109.50,214.18,205.73,221.00,1,0.009686
1,ENGCASING & FUNNEL,98.00,12.00,6.75,17.05,2,0.008668
2,ACCOM.ST.+EQUIP,630.63,23.00,17.05,28.25,2,0.000000
3,NO.1 HATCH COAMING & COVER,82.86,194.86,186.59,203.12,2,0.000000
4,NO.2 HATCH COAMING & COVER,89.91,170.50,162.23,178.76,2,0.000000


## 钢料估算

In [7]:
# 钢料数据 
# 注意要深度拷贝，设置deep=True
data_steel = data_steel_parent.copy(deep=True)
data_steel.head()

,ITEM,W,LCG,AFTEXT,FWD EXT
0,STF-6~F11+.3,260.21,3.59,-4,8.55
1,STF11+.3~24+.35,323.26,14.28,8.55,19.00
2,ST.F24+.35~F36+.3,507.47,24. 60,1900,28.55
3,STF36+.3~F64,820.69,40.78,28.55,52.61
4,ST. F64~F214,4889.31,119.23,52.61,183.11


利用平方模数法进行设计船钢料重量估计  
$$W_H = C_{H1}L(B+D)$$

In [8]:
# 根据母船资料计算系数CH1
CH1 = data_steel_parent.iloc[:,1]/(Lpp_parent*(B_parent+D_parent))
CH1

0     0.023016
1     0.028593
2     0.044887
3     0.072592
4     0.432473
5     0.074499
6     0.046730
7     0.001021
8     0.002944
9     0.003541
10    0.003419
11    0.003644
12    0.003412
13    0.002469
14    0.000815
15    0.014084
16    0.013267
17    0.014022
18    0.013889
19    0.014098
20    0.015431
21    0.002919
Name: W, dtype: float64

In [9]:
# 计算设计船钢料重量
data_steel.iloc[:,1] = CH1*Lpp*(B+D)
data_steel.iloc[:,1] = data_steel.iloc[:,1].round(2)
data_steel

,ITEM,W,LCG,AFTEXT,FWD EXT
0,STF-6~F11+.3,253.46,3.59,-4,8.55
1,STF11+.3~24+.35,314.88,14.28,8.55,19.00
2,ST.F24+.35~F36+.3,494.31,24. 60,1900,28.55
3,STF36+.3~F64,799.41,40.78,28.55,52.61
4,ST. F64~F214,4762.56,119.23,52.61,183.11
5,STF214~F240-.3,820.42,194.17,183.11,205.43
6,ST.240-.3~FWD,514.60,211.49,205.43,221.00
7,DK F36~F41,11.24,30.43,28. 25,32.60
8,DK .F60~F70,32.42,53.48,49.13,57.83
9,DK. F89~F100,38.99,79.15,74.36,83.93


In [10]:
WH = data_steel.iloc[:,1].sum()
print("设计船的钢料重量为:",WH,'t')

设计船的钢料重量为: 9159.7 t


## 舾装与机电设备重量估算

In [11]:
data_outfitting = data_outfitting_parent.copy(deep=True)
data_outfitting.head()

,ITEM,W,LCG,AFT EXT,FWD EXT,类别,系数
0,F'CLE,109.50,214.18,205.73,221.00,1,0.009686
1,ENGCASING & FUNNEL,98.00,12.00,6.75,17.05,2,0.008668
2,ACCOM.ST.+EQUIP,630.63,23.00,17.05,28.25,2,0.000000
3,NO.1 HATCH COAMING & COVER,82.86,194.86,186.59,203.12,2,0.000000
4,NO.2 HATCH COAMING & COVER,89.91,170.50,162.23,178.76,2,0.000000


In [12]:
# 根据子船与母船的换算方法换算系数
# 与尺寸相关的利用平方模数法换算
# 其余的与母船一致,系数为0
for i in range(data_outfitting.shape[0]):
    if data_outfitting.iloc[i,6] != 0:
        data_outfitting.iloc[i,1] = data_outfitting.iloc[i,6]*Lpp*(B+D)
data_outfitting.iloc[:,1] = data_outfitting.iloc[:,1].round(2)
data_outfitting

,ITEM,W,LCG,AFT EXT,FWD EXT,类别,系数
0,F'CLE,106.66,214.18,205.73,221.00,1,0.009686
1,ENGCASING & FUNNEL,95.46,12.00,6.75,17.05,2,0.008668
2,ACCOM.ST.+EQUIP,630.63,23.00,17.05,28.25,2,0.000000
3,NO.1 HATCH COAMING & COVER,82.86,194.86,186.59,203.12,2,0.000000
4,NO.2 HATCH COAMING & COVER,89.91,170.50,162.23,178.76,2,0.000000
5,NO.3 HATCH COAMING & COVER,91.12,144.40,136.13,152.66,2,0.000000
6,NO.4 HATCH COAMING & COVER,106.92,118.30,110.03,126.56,2,0.000000
7,NO.5 HATCH COAMING & COVER,91.12,92.20,83.93,100.46,2,0.000000
8,NO.6HATCH COAMING & COVER,91.12,66.10,57.83,74.36,2,0.000000
9,NO.7 HATCH COAMING & COVER,91.12,40.87,32.60,49.13,2,0.000000


In [14]:
# 机电设备与舾装重量
WM_and_WO = data_outfitting.iloc[:,1].sum()
print("机电设备的重量为:",WM_and_WO,'t')

机电设备的重量为: 3276.4 t


## 空船重量

In [15]:
# 空船重量 考虑5%LW的排水量裕度，参考课本p67
LW = (1+0.005)*(WH + WM_and_WO)
LW = round(LW,2)
print("设计船的空船重量为%s t"%LW)

设计船的空船重量为12498.28 t


## 载重量成分计算
设计船的载重量为72100t，各组成成分量为

### 人员及行李

In [16]:
# 人员数据
# 参考课本p68,船员行李为35-55kg，此处取45kg/人
luggage_weight_per = 45
# 船员人数取为28人
crew_number = 28

In [17]:
# 行李总重量: t
luggage_weight_sum = luggage_weight_per * crew_number / 1000 # kg -> t
# 人员重量，取每人65kg，参考课本p68
person_weight_per = 65
person_weight = person_weight_per * crew_number/1000
# 人员与行李总重量
person_luggage_weight = person_weight + luggage_weight_sum
print(f'''
人员总重量为       {person_weight}t
行李总重量         {luggage_weight_sum}t
人员与行李总重量   {person_luggage_weight}t
''')


人员总重量为       1.82t
行李总重量         1.26t
人员与行李总重量   3.08t



### 食品及淡水

In [18]:
# 参考设计书中巴拿马型散货船续航力，此处取24000kn
self_sustaining_ability = R/Vs/24
# 食物定量取3.5kg/人/天，课本p68
food_weight_per = 3.5
# 淡水定量为200kg/人/天，课本p68
water_weight_per = 200
# 总重量
water_weight = round(water_weight_per*self_sustaining_ability*crew_number/1000,2)
food_weight = round(food_weight_per*self_sustaining_ability*crew_number/1000,2)
water_food_weight = round(water_weight + food_weight,2)
print(f"""
食物重量为            {food_weight}t
淡水重量为            {water_weight}t
食物与淡水总重量为    {water_food_weight}t
""")


食物重量为            6.76t
淡水重量为            386.21t
食物与淡水总重量为    392.97t



### 油水

公式参考课本p69
$$W_F = g_0P_1t\cdot k\cdot 10^{-3}\\
W_L=\epsilon_1 W_F\\
W_{BW}=G\cdot \epsilon_2 \cdot t
$$
$\epsilon_1$取0.04,$\epsilon_2$取0.05，参考课本p69

In [19]:
## 燃油储备量
# 主机采用MAN-B&W 6S50ME-C主机，燃油消耗率为0.171 kg/(kW.h)，主机常用额定功率参考母型船总设计船中的8058kW
# g0取主机耗油率的1.17，k取1.15，参考课本p69
g0 = 0.171*1.17
t = R/Vs # h
P1 = 8058
k = 1.15
WF = round(g0*P1*t*k/1000,2) # kg->t
## 润滑油储备量
epsilon1 = 0.04
WL = round(epsilon1*WF,2)
#炉水储备量,参考任务书母船p89，取Other Tank的重量81t
epsilon2 = 0.05
WBW = 81
print(f'''
燃油储备量为     {WF}t
润滑油储备量为   {WL}t
炉水储备量为     {WBW}t
''')


燃油储备量为     3068.67t
润滑油储备量为   122.75t
炉水储备量为     81t



### 备品、供应品重量

In [20]:
# 参考课本p69，备品与供应品的重量为0.5%-1%LW，此处取0.75%
spares_weight = 0.75/100*LW
spares_weight = round(spares_weight,2)
print("备品与供应品储存量为:%s t"%spares_weight)

备品与供应品储存量为:93.74 t


### 载货量

In [21]:
# 载货量为载重量DW-人员及行李重量-食品及淡水重量-燃油重量-润滑油重量-炉水重量-备品与供应品重量
goods_weight = DW - person_luggage_weight - water_food_weight -WF - WL - WBW - spares_weight
print("载货量为%s t"%goods_weight)

载货量为68337.79 t


## 重量校核

In [22]:
# 此处由于与excel中CH1的保留小数位数差异问题，钢料重量
# 比excel中小一点，因此最终空船重量也小了一些,约7t
W = round(LW + DW,2)
print(f'''
设计船重量为     {W}t
满足重力小于浮力 {W<delta}
''')


设计船重量为     84598.28t
满足重力小于浮力 False



In [23]:
delta_DW = round(W - delta,2)
print(f'''
需增加的排水量为{delta_DW}t
''')


需增加的排水量为270.79t



## 第一次迭代

总重量比排水量大，因此设计船的载重量比预期载重量小，利用诺曼系数法进行主尺度修正，这里修正吃水  
$$N = \frac{1}{1-[\alpha \frac{W_H}{\triangle}+
\beta\frac{W_O}{\triangle}+\gamma\frac{W_M}{\triangle}]}\\
\delta \triangle = N\cdot \delta DW
$$

In [46]:
# 参考课本p73
alpha = 0.4
beta = gamma = 0

In [48]:
N = 1/(1-alpha*WH/delta)
ddelta = round(N*delta_DW,2)
print("排水量变化值为%s t"%ddelta)

排水量变化值为283.09 t


In [49]:
# 结构吃水处的水线面系数参考母船数据0.929，指导书p83
C_W = 0.929
# 水线面处的面积为
A_W = Lpp*B*C_W
# 排水量变化后的结构吃水变化为
dd_structure = round(ddelta/rho/A_W,2)
print("结构吃水变化值为:%sm"%dd_structure)

结构吃水变化值为:0.04m


In [51]:
# 第一次迭代后的尺寸为
d_structure = round(d+dd_structure,2)
print(f'''
第一次迭代后的尺寸为:
结构吃水:     {d_structure}m
船长:         {Lpp}m
型宽:         {B}m
型深:         {D}m
''')


第一次迭代后的尺寸为:
结构吃水:     14.15m
船长:         215.17m
型宽:         31.84m
型深:         19.34m



In [53]:
# 最终的排水量为
delta_final = delta + ddelta
print("迭代后的排水量为:",delta_final,'t')
print("满足重力与浮力平衡：",delta_final>=W)

迭代后的排水量为: 84610.58 t
满足重力与浮力平衡： True
